In [3]:
import pandas as pd
import os, shutil
import tensorflow as tf
import IPython.display as display
import numpy as np
import time
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score, roc_curve, auc
from sklearn.metrics import plot_confusion_matrix, classification_report
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time

# Load in Images

The data directories are comprised of the file path strings, one for each train, test, and split. Within the file paths are two folders. One folder has normal chest xray photos (no pneumonia) and another folder has chest xrays of people with pneumonia.

In [ ]:
datadir_train = 'C:/Users/Raizel/Desktop/Flatiron/phase1/Project4/PneumoniaClassification/chest_xray/train'
datadir_test = 'C:/Users/Raizel/Desktop/Flatiron/phase1/Project4/PneumoniaClassification/chest_xray/test'
datadir_validate = 'C:/Users/Raizel/Desktop/Flatiron/phase1/Project4/PneumoniaClassification/chest_xray/val'
categories = ['NORMAL', 'PNEUMONIA']

# Pneumonia Domain Knowledge

According to the MayoClinic, "Pneumonia is an infection that inflames the air sacs in one or both lungs." People with Pneumonia experience caouing, extreme phlegm, fever, difficulty breathing, and chills. Pneumonia, like many other illness, is very uncomfortable. Using neural networks modeling once can classify an image of a healthy lung compared to the image of a lung of a Pneumonia patient. This way the doctor can learn more about the disease to correctly classify patients as having Pneumonia or not having Pneumonia.

# Data Augmentation

In [ ]:
data_train = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        datadir_train, 
        target_size=(256, 256), 
        batch_size = 600, 
        seed = 27,
        class_mode="binary") 

In [ ]:
data_test = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        datadir_test, 
        target_size=(256, 256), 
        batch_size = 624, 
        seed = 27,
        class_mode="binary")

In [ ]:
data_val = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        datadir_validate, 
        target_size=(256, 256), 
        batch_size = 16, 
        seed = 12,
        class_mode="binary")

# Images

In [ ]:
train_images, train_labels = next(data_train)
test_images, test_labels = next(data_test)
val_images, val_labels = next(data_val)

In [ ]:
X_train = np.array(train_images)
y_train = np.array(train_labels)

X_test = np.array(test_images)
y_test = np.array(test_labels)

X_val = np.array(val_images)
y_val = np.array(val_labels)

In [ ]:
#visualize images
%matplotlib inline
import matplotlib.pyplot as plt


fig = plt.figure(figsize = (18,12))
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(train_images[i])
    plt.title(categories[int(train_labels[i])], fontsize = 15)
plt.show()

Since Pneumonia is known to inflame both lungs, I predicted the cloudier images are from Pneumonia patients. https://www.radiologyinfo.org/en/info.cfm?pg=pneumonia says white spots in the lungs is how doctors find an infection from an chest x-ray. 

# Model One - Feedforward Neural Network

In [ ]:
num_classes = 1

# specifying the model structure
model = Sequential()

#matching shape sizes
print(y_train.shape)
print(X_train.shape)

# specify the first hidden layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# specify the second layer
model.add(Dense(64))
model.add(Dropout(0.2))

# specify the output layer
model.add(Dense(num_classes, activation='sigmoid'))

# model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

#history necessary for summary and evaluation
history = model.fit(X_train.flatten()[:len(y_train)], y_train,
                    batch_size= 1,
                    epochs=4,
                    verbose=1,
                    validation_data=(X_val.flatten()[:len(y_val)], y_val.flatten()))
score = model.evaluate(X_test.flatten()[:len(y_test)], y_test.flatten(), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.summary()

In [ ]:
results_train = model.evaluate(X_train.flatten()[:len(y_train)], y_train)
results_test = model.evaluate(X_test.flatten()[:len(y_test)], y_test.flatten())
print(results_train, results_test)

# Evaluation Metrics - Model One

In [ ]:
#confusion matrix
pred = cnn.predict(X_test)
cm = confusion_matrix(y_test, pred.round())
cm

In [ ]:
classification_report(y_test, pred.round())

In [ ]:
sns.heatmap(cm, annot=True, fmt='g', xticklabels = ['normal', 'pn'], yticklabels = ['normal', 'pn'])

ex. brain surgeoun trying to move cancer - precsion - being conservative and not moving a lot of cells, just cancerous cells - recall - get rid of more cells assuming cancerous - not leaving positives behind

Precision = TP/TP+FP Recall = TP/TP+FN

advantages
- independent, observations don't imapact each other

disadvantages
- underfit ---> variance lower , rigid structure - very few parameters - simple model therefore less variance 
- uses gradient descent - cost function is log likely hood function, if you have a lot of parameters very hard for a lot of parameters to converge, might get stuck on local maximum bc dont have global maximum 

In [ ]:
There are 75 true positive (normal images), 371 true negative (pneumonia images), 